# Syntactic score functions

## Yes/No question classification

In [7]:
def is_yes_no_question(question):
    question = question.lower().strip()
    if question.startswith(('is', 'are', 'do', 'does', 'did', 'was', 'were', 'will', 'can', 'could', 'should', 'have', 'has', 'had')):
        return 1
    else:
        return 0

def is_yes_no_answer(answer):
    answer = answer.lower().strip()
    if answer.startswith(('yes','no')):
        return 1
    else:
        return 0

In [35]:
# Test the function
sentences = [
    "Is this right?",
    "Which questions are you thinking of?",
    "Fox."
]

for sentence in sentences:
    is_yes_no = is_yes_no_question(sentence)
    print(f"Sentence: {sentence}")
    print(f"Is yes/no type question: {is_yes_no:.2f}\n")

Sentence: Is this right?
Is yes/no type question: 1.00

Sentence: Which questions are you thinking of?
Is yes/no type question: 0.00

Sentence: Fox.
Is yes/no type question: 0.00



## Conciseness

In [9]:
import spacy
from nltk.tokenize import word_tokenize

# Load the spaCy model for linguistic features
nlp = spacy.load("en_core_web_sm")

def evaluate_conciseness(answer):
    # Tokenize the answer and analyze with spaCy
    doc = nlp(answer)
    word_count = len(word_tokenize(answer))

    # Calculate the ratio of content words (nouns, verbs, adjectives, adverbs) to total words
    content_words_count = sum(token.pos in [spacy.symbols.NOUN, spacy.symbols.VERB, spacy.symbols.ADJ, spacy.symbols.ADV] for token in doc)
    content_ratio = content_words_count / word_count if word_count else 0

    # Conciseness favors higher content ratio (more information with fewer words)
    conciseness_score = content_ratio

    # Adjusting score for extremely short answers
    # Extremely short answers (like 'yes' or 'no') are typically very concise
    if word_count <= 2:
        conciseness_score = max(conciseness_score, 0.9)

    # Adjusting score to be between 0 and 1
    conciseness_score = max(0, min(conciseness_score, 1))

    return conciseness_score

In [10]:
# Test the function
answers = [
    "Yes.",
    "The cat sat on the mat.",
    "This is a somewhat more elaborative answer providing detailed information, albeit not necessarily in a concise manner."
]

for answer in answers:
    score = evaluate_conciseness(answer)
    print(f"Answer: {answer}\nConciseness Score: {score:.2f}\n")

Answer: Yes.
Conciseness Score: 0.90

Answer: The cat sat on the mat.
Conciseness Score: 0.43

Answer: This is a somewhat more elaborative answer providing detailed information, albeit not necessarily in a concise manner.
Conciseness Score: 0.53



## Syntactic score

In [28]:
def syntactic_score(question,answer, debug=False):
    coherency = 1 if is_yes_no_question(question) == is_yes_no_answer(answer) else 0
    fluency=evaluate_syntax_fluency(answer)
    conciseness=evaluate_conciseness(answer)
    syntactic_score=(coherency+fluency+conciseness)/3

    if debug==True:
        print('Q:',question,'|YES/NO?',is_yes_no_question(question), '||A:',answer,'|COHERENCY:',coherency, '|FLUENCY:', fluency, '|CONCISENESS:',conciseness,'||SYNTACTIC SCORE:',syntactic_score , '\n' ) 

    return syntactic_score

In [12]:
# Examples putting it all together
questions=['Are dogs pets?',
           'Is a dog a pet?',

           'Is it Saturday',
           'What day is it?',

           'Where are you going?',
           'Are you goind downtown?',

           'Did you forget?',
           'Have you forgotten?',

           'How long has it been?',
           'Has it been long?'
           ]

answers=['I dont know',
         'Yes it is',

         'It is not Saturday',
         'No',

         'Nowhere, stop asking',
         'YES!',
         
         'Maybe I did',
         'No. Stop bothering',

         'Its been very long since we last spoke',
         'No, not really']

In [29]:
# Test the function
for i in range(len(questions)):
    syntactic_score_=syntactic_score(questions[i],answers[i],debug=True)

Q: Are dogs pets? |YES/NO? 1 ||A: I dont know |COHERENCY: 0 |FLUENCY: 0.9375 |CONCISENESS: 0.3333333333333333 ||SYNTACTIC SCORE: 0.4236111111111111 

Q: Is a dog a pet? |YES/NO? 1 ||A: Yes it is |COHERENCY: 1 |FLUENCY: 0.9166666666666667 |CONCISENESS: 0 ||SYNTACTIC SCORE: 0.638888888888889 

Q: Is it Saturday |YES/NO? 1 ||A: It is not Saturday |COHERENCY: 0 |FLUENCY: 0.9375 |CONCISENESS: 0 ||SYNTACTIC SCORE: 0.3125 

Q: What day is it? |YES/NO? 0 ||A: No |COHERENCY: 0 |FLUENCY: 0.5 |CONCISENESS: 0.9 ||SYNTACTIC SCORE: 0.4666666666666666 

Q: Where are you going? |YES/NO? 0 ||A: Nowhere, stop asking |COHERENCY: 0 |FLUENCY: 0.9375 |CONCISENESS: 0.75 ||SYNTACTIC SCORE: 0.5625 

Q: Are you goind downtown? |YES/NO? 1 ||A: YES! |COHERENCY: 1 |FLUENCY: 0.5 |CONCISENESS: 0.9 ||SYNTACTIC SCORE: 0.7999999999999999 

Q: Did you forget? |YES/NO? 1 ||A: Maybe I did |COHERENCY: 0 |FLUENCY: 0.9166666666666667 |CONCISENESS: 0.6666666666666666 ||SYNTACTIC SCORE: 0.5277777777777778 

Q: Have you forgott

## Fluency

In [5]:
!pip install stanza
import spacy
import torch
import stanza

# Download and set up the Stanza pipeline
stanza.download('en')  # for English
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def evaluate_syntax_fluency(sentence):
    doc = nlp(sentence)

    if len(doc) == 0:
        return 0.0  # Empty sentence

    num_tokens = len(doc)
    if num_tokens <= 2:  # Penalize very short or fragmentary sentences
        return 0.5

    tree_depths = []
    unique_dependency_types = set()

    for token in doc:
        # Calculate depth of each token in the parse tree
        depth = 0
        current_token = token
        while current_token.head != current_token:
            depth += 1
            current_token = current_token.head
        tree_depths.append(depth)

        # Collect unique dependency types
        unique_dependency_types.add(token.dep_)

    # Metrics
    max_depth = max(tree_depths)
    depth_variety_score = len(unique_dependency_types) / num_tokens

    # Score calculation (adjusted heuristic)
    # Higher max depth might indicate complexity (lower fluency)
    # More variety in dependency types might indicate richer syntactic structure (higher fluency)
    fluency_score = (1 - (max_depth / (2 * num_tokens)) + depth_variety_score) / 2

    return fluency_score


In [37]:
# Test the function
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "While the fox jumps, the dog barks.",
    "Fox."
]

for sentence in sentences:
    fluency_score = evaluate_syntax_fluency(sentence)
    print(f"Sentence: {sentence}")
    print(f"Syntax Fluency Score: {fluency_score:.2f}\n")

Sentence: The quick brown fox jumps over the lazy dog.
Syntax Fluency Score: 0.77

Sentence: While the fox jumps, the dog barks.
Syntax Fluency Score: 0.81

Sentence: Fox.
Syntax Fluency Score: 0.50



# QA Dataset application

In [15]:
'''
QA dataset line:
- [0] article_name   
- [1] question   
- [2] answer  
- [3] difficulty by question maker
- [4] difficulty by answer maker
- [5] article path
- [6] article text
'''

'\nQA dataset line:\n- [0] article_name   \n- [1] question   \n- [2] answer  \n- [3] difficulty by question maker\n- [4] difficulty by answer maker\n- [5] article path\n- [6] article text\n'

In [16]:
# Change to appropriate local file path
%cd '/content/drive/MyDrive/2023-FALL/11-611 NLP/NLP Project Ideas'

[WinError 3] The system cannot find the path specified: "'/content/drive/MyDrive/2023-FALL/11-611 NLP/NLP Project Ideas'"
h:\My Drive\2023-FALL\11-611 NLP\Project


In [20]:

import torch
from torch.utils.data import Dataset
import os
import numpy as np


class CustomData(Dataset):

    def __init__(self, file_dir):
        self.file = file_dir
        self.article_name = []
        self.questions = []
        self.answers = []
        self.q_diffi = []
        self.a_diffi = []
        self.article_path = []
        self.context = {} # only fill when load the dataset

        # get question answer pairs
        for div in ['S08', 'S09', 'S10']:
          skip = True
          qa_path = os.path.join(self.file, div, "question_answer_pairs.txt")
          with open(qa_path, 'rb') as f:
            for line in f:
              if skip:
                skip = False
                continue
              try:
                row = line.decode().split('\t')
              except:
                continue
              # print(row)
              if "NULL" in row:
                continue # if any feature does not exist -> skip
              self.article_name.append(row[0])
              self.questions.append(row[1])
              self.answers.append(row[2])
              self.q_diffi.append(row[3])
              self.a_diffi.append(row[4])
              self.article_path.append(div + "/"+ row[5][:-1]) # get rid of '\n

        print("length of dataset: ", len(self.questions))


    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):

        # retrieve context here -> less mem storage overhead
        try:
          curr_context = self.context[self.article_name[idx]]
        except KeyError:
          context_file = self.file + "/" + self.article_path[idx] + ".txt"
          # read all content, including the related items
          with open(context_file, 'rb') as f:
            curr_context = f.read().decode().replace('\n',' ')
          self.context[self.article_name[idx]] = curr_context

        #return self.questions[idx], self.answers[idx], curr_context
        return (self.article_name[idx],
                self.questions[idx],
                self.answers[idx],
                self.q_diffi[idx],
                self.a_diffi[idx],
                self.article_path[idx],
                curr_context
                )


In [21]:
# Load data using CustomData function
data_path = 'Question_Answer_Dataset_v1.2'
dataset = CustomData(data_path)
# Print random set for data exploration
n=len(dataset)
n_show=2

rand_range=np.random.randint(0,n,n_show)

for line in rand_range:
  d = dataset[line] #
  print('line:',line)
  print("article_name: ", d[0])
  print("question: ", d[1])
  print(' question length:',len(d[1]))
  print("answer: ", d[2])
  print(' answer length:', len(d[2]))
  print("q_diffi: ", d[3])
  print("a_diffi: ", d[4])
  print("article_path: ", d[5])
  print("article: ", d[6])
  print(' article length:',len(d[6]))
  print('\n')

length of dataset:  2725
line: 2561
article_name:  Swedish_language
question:  Does Swedish use the perfect participle to form the present perfect tense?
 question length: 74
answer:  No
 answer length: 2
q_diffi:  easy
a_diffi:  easy
article_path:  S10/data/set5/a9
article:  Swedish_language    Swedish ( ) is a North Germanic language, spoken by approximately 10 million people   gives the number of 8,789,835, but is based on data from 1986. Sweden has currently a population of 9.2 Mio (2008 census), and there are about 290,000 native speakers of Swedish in Finland  , based on data from 2007), leading to an estimate of about 9 to 10 Mio. , predominantly in Sweden and parts of Finland, especially along the coast and on the Åland islands. It is to a considerable extent mutually intelligible with Norwegian and to a lesser extent with Danish (see especially "Classification").  Along with the other North Germanic languages, Swedish is a descendant of Old Norse, the common language of the Ge

In [19]:
print(dataset[439])

('Gray_Wolf', 'Do wolves leave their pack?', 'They leave their birth pack when they reach sexual maturity', 'easy', 'medium', 'S08/data/set1/a6', 'Gray Wolf  :For other uses, see Wolf (disambiguation), Gray Wolves (disambiguation), or Timber Wolf (comics).  The gray wolf (Canis lupus), also known as the timber wolf or, simply, wolf, is a mammal of the order Carnivora. The gray wolf is the largest member of the family Canidae and also the most well known of wolves. Its shoulder height ranges from 0.6 to 0.9 meters (26–36 inches) and its weight typically varies between 32 and 62 kilograms (70–135 pounds). As evidenced by studies of DNA sequencing and genetic drift the gray wolf shares a common ancestry with the domestic dog (Canis lupus familiaris).  Though once abundant over much of North America and Eurasia, the gray wolf inhabits a very small portion of its former range because of widespread destruction of its habitat; in some regions it is endangered or threatened. Considered as a wh

In [34]:
'''
- [0] article_name   
- [1] question   
- [2] answer  
- [3] difficulty by question maker
- [4] difficulty by answer maker
- [5] article path
- [6] article text
'''

n=50 #len(dataset) #number of questions related to articles withe difficulaty evaluations

scores=[]
for line in range(n):
    q=dataset[line][1]
    a=dataset[line][2]
    print('difficulty:',dataset[line][3],dataset[line][4])
    scores.append(syntactic_score(q,a,debug=True))

print(scores)

difficulty: easy easy
Q: Was Abraham Lincoln the sixteenth President of the United States? |YES/NO? 1 ||A: yes |COHERENCY: 1 |FLUENCY: 0.5 |CONCISENESS: 0.9 ||SYNTACTIC SCORE: 0.7999999999999999 

difficulty: easy easy
Q: Was Abraham Lincoln the sixteenth President of the United States? |YES/NO? 1 ||A: Yes. |COHERENCY: 1 |FLUENCY: 0.5 |CONCISENESS: 0.9 ||SYNTACTIC SCORE: 0.7999999999999999 

difficulty: easy medium
Q: Did Lincoln sign the National Banking Act of 1863? |YES/NO? 1 ||A: yes |COHERENCY: 1 |FLUENCY: 0.5 |CONCISENESS: 0.9 ||SYNTACTIC SCORE: 0.7999999999999999 

difficulty: easy easy
Q: Did Lincoln sign the National Banking Act of 1863? |YES/NO? 1 ||A: Yes. |COHERENCY: 1 |FLUENCY: 0.5 |CONCISENESS: 0.9 ||SYNTACTIC SCORE: 0.7999999999999999 

difficulty: easy medium
Q: Did his mother die of pneumonia? |YES/NO? 1 ||A: no |COHERENCY: 1 |FLUENCY: 0.5 |CONCISENESS: 0.9 ||SYNTACTIC SCORE: 0.7999999999999999 

difficulty: easy easy
Q: Did his mother die of pneumonia? |YES/NO? 1 ||A: